### Imports

In [1]:
import numpy as np
from pandas import to_datetime

from numpy import exp, where, arange
from bond_pricing import bond_duration, bond_price, edate, equiv_rate, bond_coupon_periods

### Standard Inputs

In [2]:
coupon = 0.032
par = 100
ytm = 0.032
settle = to_datetime('2025-10-15')
mat = to_datetime('2030-10-15')
freq = 2
daycount = 'actual/360'
coupon_periods = bond_coupon_periods(settle=settle, mat=mat, freq=freq, daycount=daycount)
n = coupon_periods['n']
accrual_fraction = coupon_periods['accrual_fraction']

### Approximate Macaulay & Modified Duration

Approximate Modified Duration Calculation:
$$\text{Approx. Ann. Mod. Duration}=\frac{\left(PV_{-}\right)-\left(PV_{+}\right)}{2\times\left(\Delta \text{Yield}\right)\times\left(PV_0\right)}$$

Approximate Macaulay Duration
$$\text{Ann. Mac. Duration}=\text{Approx. Ann. Mod. Duration}\times\left(1+r\right)$$

In [3]:
def approx_duration(settle, cpn, mat, yld, freq, face, daycount, delta_yld, modified=True):
    ytm_low = ytm - delta_yld
    ytm_high = ytm + delta_yld
    price = bond_price(settle=settle, cpn=cpn, mat=mat, yld=yld, freq=freq, face=face, daycount=daycount)
    price_high = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_low, freq=freq, face=face, daycount=daycount)
    price_low = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_high, freq=freq, face=face, daycount=daycount)
    
    duration = (price_high - price_low) / (2 * delta_yld * price)
    
    if modified:
        return duration
    else:
        return duration * (1 + yld/freq)

delta = 0.0005
modified = approx_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, daycount=daycount, delta_yld=delta, modified=True)
macaulay = approx_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, daycount=daycount, delta_yld=delta, modified=False)
print(f'{modified = :0.4f}')
print(f'{macaulay = :0.4f}')

modified = 4.5868
macaulay = 4.6602


### Macaulay & Modified Duration

**Macaulay Duration** complete formula:

$$MacDur=\left\lbrace\left(1-\frac{t}{T}\right)\left\lbrack\frac{\frac{PMT}{\left(1+r\right)^{1-\frac{t}{T}}}}{PV^{Full}}\right\rbrack+\left(2-\frac{t}{T}\right)\left\lbrack\frac{\frac{PMT}{\left(1+r\right)^{2-\frac{t}{T}}}}{PV^{Full}}\right\rbrack+\cdots+\left(N-\frac{t}{T}\right)\left\lbrack\frac{\frac{PMT+FV}{\left(1+r\right)^{N-\frac{t}{T}}}}{PV^{Full}}\right\rbrack\right\rbrace$$

- *t* is the number of days from the last coupon payment to the settlement date;
- *T* is the number of days in the coupon period;
- *t*/*T* is the fraction of the coupon period that has passed since the last payment;
- *PMT* is the coupon payment per period;
- *FV* is the future value paid at maturity, or the par value of the bond;
- *r* is the yield-to-maturity per period; and
- *N* is the number of evenly spaced periods to maturity as of the beginning of the current period.
- $PV^{Full}$ is the "dirty" or invoice price of the bond inclusive of accrued interest
- Annualized MacDur = MacDur / number of coupon periods per year

Simple interpretation: MacDur = Sum(Present value of the cash flow x Time to Receipt) / Present value of the bond

$$MacDur=\left\lbrace\frac{1+r}{r}-\frac{1+r+\left\lbrack N\times\left(c-r\right)\right\rbrack}{c\times\left\lbrack\left(1+r\right)^{N}-1\right\rbrack +r}\right\rbrace-\frac{t}{T}$$

- *r* is the yield-to-maturity per period;
- *N* is the number of evenly spaced periods to maturity as of the beginning of the current period;
- *c* is the coupon rate per period;
- *t* is the number of days from the last coupon payment to the settlement date; and
- *T* is the number of days in the coupon period.
- $t/T$ is the 'accural fraction' or the fraction of the period that has gone by since the last coupon.

**Modified Duration** formula:
$$\text{Modified Duration}=\frac{MacDur}{\left(1+\frac{r}{m}\right)}$$
$$\text{Annualized Modified Duration}=\frac{ModDur}{Coupon\:Periods\:Per\:Year}$$

In [4]:
macdur = bond_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, modified=False, daycount=daycount)
moddur = bond_duration(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, modified=True, daycount=daycount)

print(f'{macdur = }\n{moddur = }')

macdur = 4.660146890283194
moddur = 4.586758750278734


In [5]:
r = ytm/freq
N = n
c = coupon/freq
tT = accrual_fraction

dur = ((1 + r)/r) - ((1 + r + N * (c - r))/(c * ((1 + r)**N - 1) + r)) - tT
macdur = dur/freq
moddur = macdur/(1 + r)

print(f'{macdur = }')
print(f'{moddur = }')

macdur = 4.660146890283194
moddur = 4.586758750278734


In [6]:
# consider a bond which has just paid a coupon
# decompose it as a portfolio of three positions
# Position                 PV           Duration_minus_(1+y)/y
# --------                 --           ----------------------
# perpetuity of c       c/y = cF/Fy                0
# short perpetuity     -c/y/F = -c/Fy              T
# Redemption of R        R/F = Ry/Fy            T-(1+y)/y
# where F is Future Value Factor (1+y)^T
# Portfolio Duration is (1+y)/y - N/D where
# N = cT - RyT + R(1+y) = R[1+y + T(c/R-y)]
# D = cF - c + Ry = R[c(F-1)/R + y]
# where Fy has been eliminated from both N and D
# Eliminating R gives
# N = 1+y + T(c/R-y)
# D = c(F-1)/R + y
# we compute duration in coupon periods as on previous coupon date
# find the equivalent yield that matches the coupon frequency

redeem = par
R = redeem / par
y = ytm / freq
c = coupon / freq
T = n
F = (1 + y)**T
dur = (1+y)/y - (1+y + T*(c/R-y)) / (c*(F-1)/R + y)
# now we subtract the fractional coupon period
dur -= accrual_fraction

# then we convert to years
dur /= freq

macdur = dur
moddur = macdur / (1+y)

print(f'{macdur = }')
print(f'{moddur = }')

macdur = 4.660146890283194
moddur = 4.586758750278734


| Period | Time to Reciept | Cash Flow | PV       | Weight       | Time x Weight |
| ------ | --------------- | --------- | -------- | ------------ | ------------- |
| 1      | 1               | 1.6       | 1.5748   | 0.0157       | 0.0157        |
| 2      | 2               | 1.6       | 1.5500   | 0.0155       | 0.0310        |
| 3      | 3               | 1.6       | 1.5256   | 0.0153       | 0.0458        |
| 4      | 4               | 1.6       | 1.5016   | 0.0150       | 0.0601        |
| 5      | 5               | 1.6       | 1.4779   | 0.0148       | 0.0739        |
| 6      | 6               | 1.6       | 1.4546   | 0.0145       | 0.0873        |
| 7      | 7               | 1.6       | 1.4317   | 0.0143       | 0.1002        |
| 8      | 8               | 1.6       | 1.4092   | 0.0141       | 0.1127        |
| 9      | 9               | 1.6       | 1.3870   | 0.0139       | 0.1248        |
| 10     | 10              | 101.6     | 86.6875  | 0.8669       | 8.6688        |
|        |                 |           | 100.0000 | 1.0000       | 9.3203        |
|        |                 |           |          | Mac Duration | 4.6601        |
|        |                 |           |          | Mod Duration | 4.5868        |

In [19]:
from prettytable import PrettyTable
from importlib import import_module

daycount_dict = {'actual/360': 'actual360', 
                 'actual/365': 'actual365', 
                 'actual/actual': 'actualactual', 
                 'thirty/360': 'thirty360'}
name = 'isda_daycounters.'+ daycount_dict[daycount]

# Accrual Fraction & Number of evenly spaced periods to maturity as of the beginning of the current period
# approximate number of full coupon periods left
n = int(freq * (mat - settle).days / 360)
# the divisor of 360 guarantees this is an overestimate
# we keep reducing n till it is right
while (edate(mat, -n * 12 / freq) <= settle):
    n -= 1
next_coupon = edate(mat, -n * 12 / freq)
n += 1  # n is now number of full coupons since previous coupon
prev_coupon = edate(mat, -n * 12 / freq)

discounting_fraction = import_module(name).year_fraction(settle, next_coupon) * freq
accrual_fraction = import_module(name).year_fraction(prev_coupon, settle) * freq

if accrual_fraction == 1:
    # We are on coupon date. Assume that bond is ex-interest
    # Remove today's coupon
    discounting_fraction += 1
    accrual_fraction -= 1

print(f"""{n = },
{discounting_fraction = },
{accrual_fraction = },
{next_coupon = },
{prev_coupon = }
""")

# Time to Reciept of Cash Flows
add = 1 if prev_coupon >= settle else 0
cf_t = arange(start=(1 - accrual_fraction), stop=n+add, step=1)

# Bond Cash Flows
cf = np.full(n, coupon/freq * par)
cf[n-1] += par

# Present Value of Cash Flows
pv_cf = cf/((1+ytm/freq)**cf_t)

# Time Weighted Present Value of Cash Flows
cf_sum = np.sum(pv_cf)
weight = pv_cf / cf_sum
time_weighted = weight * cf_t

# Duration computations
macdur = np.sum(time_weighted) / freq
moddur = macdur / (1 + ytm / freq)

# Output Table
table = PrettyTable()
table.add_column("Period", arange(1,n+1))
table.add_column("Time to Receipt", np.round(cf_t, 4))
table.add_column("Cash Flow", cf)
table.add_column("PV", np.round(pv_cf, 4))
table.add_column("Weight", np.round(weight, 4))
table.add_column("Time Weighted", np.round(time_weighted, 4))
print(table)

print(f'{macdur = }')
print(f'{moddur = }')

n = 10,
discounting_fraction = 1.011111111111111,
accrual_fraction = 0.0,
next_coupon = Timestamp('2026-04-15 00:00:00'),
prev_coupon = Timestamp('2025-10-15 00:00:00')

+--------+-----------------+-----------+---------+--------+---------------+
| Period | Time to Receipt | Cash Flow |    PV   | Weight | Time Weighted |
+--------+-----------------+-----------+---------+--------+---------------+
|   1    |       1.0       |    1.6    |  1.5748 | 0.0157 |     0.0157    |
|   2    |       2.0       |    1.6    |   1.55  | 0.0155 |     0.031     |
|   3    |       3.0       |    1.6    |  1.5256 | 0.0153 |     0.0458    |
|   4    |       4.0       |    1.6    |  1.5016 | 0.015  |     0.0601    |
|   5    |       5.0       |    1.6    |  1.4779 | 0.0148 |     0.0739    |
|   6    |       6.0       |    1.6    |  1.4546 | 0.0145 |     0.0873    |
|   7    |       7.0       |    1.6    |  1.4317 | 0.0143 |     0.1002    |
|   8    |       8.0       |    1.6    |  1.4092 | 0.0141 |     0.1127

In [8]:
cc_rate = equiv_rate(ytm/freq, from_freq=freq, to_freq=np.inf)
print(cf_t)
cf_ta = np.arange(1, n+1)
print(cf_ta)
df = exp(-cc_rate * cf_ta)
print(cf)
D = np.dot(cf*df, cf_ta) / np.dot(cf, df) - accrual_fraction
    
D /= where(False, freq/(1 + ytm/freq), freq)

print(D)

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[ 1  2  3  4  5  6  7  8  9 10]
[  1.6   1.6   1.6   1.6   1.6   1.6   1.6   1.6   1.6 101.6]
4.660024193738399


### Approximate Convexity

Approximate Convexity Formula:

$$\text{Approx. Con.}=\frac{\left(PV_{-}\right)+\left(PV_{+}\right)-\left\lbrack2\times\left(PV_0\right)\right\rbrack}{\left(\Delta Yield\right)^2\times\left(PV_0\right)}$$

In [9]:
def approx_con(settle, cpn, mat, yld, freq, face, daycount, delta_yld, modified=True):
    ytm_low = ytm - delta_yld
    ytm_high = ytm + delta_yld
    price = bond_price(settle=settle, cpn=cpn, mat=mat, yld=yld, freq=freq, face=face, daycount=daycount)
    price_high = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_low, freq=freq, face=face, daycount=daycount)
    price_low = bond_price(settle=settle, cpn=cpn, mat=mat, yld=ytm_high, freq=freq, face=face, daycount=daycount)
    
    return (price_high + price_low - (2 * price)) / (delta_yld**2 * price) 

delta = 0.0005
convexity = approx_con(settle=settle, cpn=coupon, mat=mat, yld=ytm, freq=freq, face=par, daycount=daycount, delta_yld=delta, modified=True)
print(f'{convexity = :0.5f}')

convexity = 24.23896


### Convexity

Calculating Convexity of Cash Flow:

$$\text{Convexity of CF}_{n}=\left(p_{n}-\frac{t}{T}\right)\left(p_{n}-\frac{t}{T}+1\right)\left(\frac{PV_{CF_{n}}}{PV^{Full}}\right)\left(1+\frac{YTM}{m}\right)^{-m}$$

$$\text{Annualized Convexity}=\frac{\sum_{n=1}^{N}\text{Convexity of CF}_{n}}{m^2}$$

- $p_{n}$ = period of cash flow
- $t/T$ = accrual fraction
- $p_{n} - t/T$ = time from settlement when coupon is due
- $\frac{PV_{CF_{n}}}{PV^{Full}}$ = present value weighted cash flow at period *n*
- m = periods per year

The bond convexity statistic is the second-order effect in the Taylor series expansion. The results are complicated enough to warrant 3 separate steps:

Step 1 - Convexity (t/T = 0):
$$Convexity(t/T=0)=\frac{\Big[2 \times c \times (1 + y)^{2} \times \Big((1+y)^{N} - \frac{1+y+(y \times N)}{1 + y}\Big) \Big] + [N \times (N + 1) \times y^{2} \times (y - c)]}{y^{2} \times (1+y)^{2} \times (c \times [(1+y)^{N}-1]+y)}$$

Step 2 - Macaulay Duration(t/T = 0):
$$MacDur=\left\lbrace\frac{1+r}{r}-\frac{1+r+\left\lbrack N\times\left(c-r\right)\right\rbrack}{c\times\left\lbrack\left(1+r\right)^{N}-1\right\rbrack +r}\right\rbrace$$

Step 3 - Convexity:
$$Convexity=Convexity(t/T=0)-\Big\lbrace\frac{t/T}{(1+y)^{2}} \times [(2\times MacDur(t/T=0)) + (1-t/T)]\Big\rbrace$$


- c = the coupon rate per period
- y = yield to maturity per period
- N = number of periods to maturity

In [10]:
c = coupon / freq
y = ytm / freq
Y = (1+y)**2
N = n
Yn = (1+y)**N
tT = accrual_fraction

con_0_num = (2 * c * Y * (Yn - ((1+y+(y*N))/(1+y)))) + (N * (N + 1) * (y**2) * (y - c))
con_0_dem = (y**2) * Y * (c * (Yn - 1) + y)
con_0 = con_0_num / con_0_dem

macdur_0 = ((1 + y)/y) - ((1 + y + (N * (c - y)))/(c * (Yn - 1) + y))

convexity = con_0 - ((tT / Y) * (( 2 * macdur_0) + (1 - tT)))

anncon = convexity/freq**2

print(f'{convexity = }')
print(f'{anncon = }')

convexity = 96.95578010034274
anncon = 24.238945025085684


In [20]:
cf_con = (cf_t * (cf_t + 1)) * weight * ((1 + ytm/freq)**-freq)
table.add_column("Convexity of CF", np.round(cf_con, 4))
print(table)
convexity = np.sum(cf_con)
anncon = convexity / freq**2

print(f'{convexity = }')
print(f'{anncon = }')

+--------+-----------------+-----------+---------+--------+---------------+-----------------+
| Period | Time to Receipt | Cash Flow |    PV   | Weight | Time Weighted | Convexity of CF |
+--------+-----------------+-----------+---------+--------+---------------+-----------------+
|   1    |       1.0       |    1.6    |  1.5748 | 0.0157 |     0.0157    |      0.0305     |
|   2    |       2.0       |    1.6    |   1.55  | 0.0155 |     0.031     |      0.0901     |
|   3    |       3.0       |    1.6    |  1.5256 | 0.0153 |     0.0458    |      0.1774     |
|   4    |       4.0       |    1.6    |  1.5016 | 0.015  |     0.0601    |      0.2909     |
|   5    |       5.0       |    1.6    |  1.4779 | 0.0148 |     0.0739    |      0.4295     |
|   6    |       6.0       |    1.6    |  1.4546 | 0.0145 |     0.0873    |      0.5919     |
|   7    |       7.0       |    1.6    |  1.4317 | 0.0143 |     0.1002    |      0.7767     |
|   8    |       8.0       |    1.6    |  1.4092 | 0.0141 | 